In [49]:
import random
import math
from statistics import mean, stdev

def is_max_die_value(dice_str, roll_value):
    num_dice, num_faces = map(int, dice_str.split('d'))
    return roll_value == num_faces    

print("Mobile Git client Test")

Mobile Git client Test


In [50]:
class DiceRoller:
    def roll(dice_str, advantage=0, print_results=False):
        # ABSTRACT INTERFACE FOR ROLLING DICE
        # SHOULD RETURN A LIST OF INTS REPRESENTING THE DICE RESULTS
        pass
        
    def roll_as_modifier(dice_str, advantage=0, print_results=False):
        # ABSTRACT INTERFACE FOR ROLLING DICE
        # SHOULD RETURN A LIST OF INTS REPRESENTING THE DICE RESULTS
        pass

In [51]:
class NimbleDiceRoller(DiceRoller):
    def roll(self, dice_str, advantage=0, print_results=False):
        try:
            # Split the input string into number of dice and number of faces
            num_dice, num_faces = map(int, dice_str.split('d'))

            if print_results:
                print(f'Rolling {dice_str}...')
            
            num_advantage_stacks = abs(advantage)
            num_dice += num_advantage_stacks

            results = [random.randint(1, num_faces) for _ in range(num_dice)]

            if print_results:
                print(f'Original roll: {results}')
                
            for _ in range(num_advantage_stacks):
                if advantage > 0:
                    results.remove(min(results))
                else:
                    results.remove(max(results))

            if print_results:
                print(f'After advantage: {results}')

            is_crit = results[0] == num_faces
            while is_crit:
                results.append(random.randint(1, num_faces))
                is_crit = results[-1] == num_faces
    
            if print_results:
                print(f'After exploding: {results}')

            return results
        
        except ValueError:
            # Handle the case where the input values are not valid
            return "Invalid input values. Please use integers for num_dice and num_faces."

    def roll_as_modifier(self, dice_str, advantage=0, print_results=False):
        try:
            # Split the input string into number of dice and number of faces
            num_dice, num_faces = map(int, dice_str.split('d'))

            if print_results:
                print(f'Rolling {dice_str}...')
            
            num_advantage_stacks = abs(advantage)
            num_dice += num_advantage_stacks

            results = [random.randint(1, num_faces) for _ in range(num_dice)]

            if print_results:
                print(f'Original roll: {results}')
                
            for _ in range(num_advantage_stacks):
                if advantage > 0:
                    results.remove(min(results))
                else:
                    results.remove(max(results))

            if print_results:
                print(f'After advantage: {results}')

            return results
        
        except ValueError:
            # Handle the case where the input values are not valid
            return "Invalid input values. Please use integers for num_dice and num_faces."

NimbleDiceRoller().roll("4d10", advantage=-2, print_results=True)


Rolling 4d10...
Original roll: [2, 9, 2, 3, 1, 10]
After advantage: [2, 2, 3, 1]
After exploding: [2, 2, 3, 1]


[2, 2, 3, 1]

In [52]:
def attack_roll(dice_str, advantage, dice_roller):
    results = dice_roller.roll(dice_str, advantage)
    if results[0] == 1:
        return 0
    return sum(results)

def get_average_damage(dice_str, advantage, dice_roller, num_tests=100000):
    average = 0
    for _ in range(num_tests):
        damage = attack_roll(dice_str, advantage, dice_roller)
        average += damage / num_tests
    return average

# print(get_average_damage("1d4", 0, NimbleDiceRoller(), 100000))

In [53]:
class Character:
    def __init__(self, name, hp, ac, num_attacks, attack_dice, damage_modifier):
        self.name = name
        self.hp = hp
        self.ac = ac
        self.num_attacks = num_attacks
        self.attack_dice = attack_dice
        self.damage_modifier = damage_modifier

In [54]:
fighter = Character(name="Fighter 3", hp=40, ac=18, num_attacks=1, attack_dice="1d8", damage_modifier=3)
monster = Character(name="Light Armor Monster 3", hp=34, ac=12, num_attacks=2, attack_dice="1d10", damage_modifier=4)

In [55]:
class AttackProcessor:
    def start_of_round(self):
        # PROCESSING AT START OF ROUND
        pass
        
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results = False):
        # ABSTRACT DEFINITION FOR PERFORMING ATTACKS
        pass

class DefenseProcessor:
    def start_of_round(self):
        # PROCESSING AT START OF ROUND
        pass
        
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        # ABSTRACT DEFINITION FOR CHECKING IF AN ATTACK HIT
        pass
        
    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        # ABSTRACT DEFINITION FOR DEFENDING ATTACKS
        pass

In [56]:
def get_damage_for_round(attacker, defender, attack_processor, defense_processor, dice_roller, print_results=False):
    attack_processor.start_of_round()
    defense_processor.start_of_round()
    damage = attack_processor.perform_attacks(attacker, defender, defense_processor, dice_roller, print_results)
    return damage

In [57]:
class NimbleStandardAttack(AttackProcessor):
    def __init__(self, advantage=0):
        self.advantage_mod = advantage

    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, 0, print_results)
            total_damage += damage

        # Process one more attack with disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, -1, print_results)
        total_damage += damage
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, attack_dice_str, damage_modifier, dice_roller, advantage, print_results=False):
        advantage += self.advantage_mod
        
        die_rolls = dice_roller.roll(attack_dice_str, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attack_dice_str}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        was_crit = is_max_die_value(attack_dice_str, die_rolls[0])
        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [58]:
class NimbleGreatWeaponFighterAttack(AttackProcessor):
    def __init__(self, advantage, gwm_advantage_threshold=0, use_total_damage=False, use_easier=False):
        self.advantage_mod = advantage
        self.gwm_advantage_threshold = gwm_advantage_threshold
        self.use_total_damage = use_total_damage
        self.use_easier = use_easier
        
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, 0, print_results)
            total_damage += damage

        # Process one more attack with disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, -1, print_results)
        total_damage += damage
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, attack_dice_str, damage_modifier, dice_roller, advantage, print_results=False):
        advantage += self.advantage_mod
        die_rolls = dice_roller.roll(attack_dice_str, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attack_dice_str}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        is_great_weapon_attack = advantage >= self.gwm_advantage_threshold
        if not is_great_weapon_attack:
            if print_results:
                print("Normal attack")
            hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)
        else:
            if print_results:
                print("Great Weapons Fighter attack")
            if self.use_total_damage:
                hit = not all([r <= 4 for r in die_rolls])
            else:
                threshold = 3 if attack_dice_str == "2d6" and self.use_easier else 4
                hit = die_rolls[0] > threshold

        was_crit = is_max_die_value(attack_dice_str, die_rolls[0])
        if hit:
            if is_great_weapon_attack:
                single_attack_die = '1' + attack_dice_str[1:]
                die_rolls += dice_roller.roll(attack_dice_str, 0, print_results)
                if print_results:
                    print("Great Weapons Fighter hit")

            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [59]:
class NimbleRogueAttack(AttackProcessor):
    def __init__(self, offhand_dice_str, sneak_attack_dice_str):
        self.offhand_dice_str = offhand_dice_str
        self.sneak_attack_dice_str = sneak_attack_dice_str
        self.sneak_attack_used = False

    def start_of_round(self):
        self.sneak_attack_used = False
        
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, 0, print_results)
            total_damage += damage

        # Process offhand attack
        damage = self.perform_single_attack(attacker, defender, defense_processor, self.offhand_dice_str, 0, dice_roller, 0, print_results)
        total_damage += damage        

        # Process one more attack with disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, -1, print_results)
        total_damage += damage        
        
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, attack_dice_str, damage_modifier, dice_roller, advantage, print_results=False):
        die_rolls = dice_roller.roll(attack_dice_str, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attack_dice_str}+{damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        was_crit = is_max_die_value(attack_dice_str, die_rolls[0])
        if hit:
            if advantage >= 0 and not self.sneak_attack_used:
                self.sneak_attack_used = True
                die_rolls += dice_roller.roll_as_modifier(self.sneak_attack_dice_str, 0, print_results)
                
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [60]:
class NimbleWizardAttack(AttackProcessor):
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, 0, print_results)
            total_damage += damage

        # Process one more attack with disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, -1, print_results)
        total_damage += damage

        # Process one more attack with 2x disadvantage
        damage = self.perform_single_attack(attacker, defender, defense_processor, attacker.attack_dice, attacker.damage_modifier, dice_roller, -2, print_results)
        total_damage += damage            
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, attack_dice_str, damage_modifier, dice_roller, advantage, print_results=False):
        die_rolls = dice_roller.roll(attack_dice_str, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attack_dice_str}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        was_crit = is_max_die_value(attack_dice_str, die_rolls[0])
        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [61]:
class NimbleMonsterLightArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls) + damage_modifier
        return damage
        
print("FIGHTER")
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(advantage=0), NimbleMonsterLightArmorDefense(), NimbleDiceRoller(), print_results=True))

print("\nROGUE")
print(get_damage_for_round(fighter, monster, NimbleRogueAttack(offhand_dice_str="1d4", sneak_attack_dice_str="2d6"), NimbleMonsterLightArmorDefense(), NimbleDiceRoller(), print_results=True))

print("\nWIZARD")
print(get_damage_for_round(fighter, monster, NimbleWizardAttack(), NimbleMonsterLightArmorDefense(), NimbleDiceRoller(), print_results=True))


FIGHTER
Rolling 1d8...
Original roll: [5]
After advantage: [5]
After exploding: [5]
Fighter 3 rolls to attack (1d8+3, adv=0): [5]
Attacker hit = True
Fighter 3 did 8 damage
Rolling 1d8...
Original roll: [2, 7]
After advantage: [2]
After exploding: [2]
Fighter 3 rolls to attack (1d8+3, adv=-1): [2]
Attacker hit = True
Fighter 3 did 5 damage
Fighter 3 total damage this round: 13
13

ROGUE
Rolling 1d8...
Original roll: [6]
After advantage: [6]
After exploding: [6]
Fighter 3 rolls to attack (1d8+3, adv=0): [6]
Attacker hit = True
Rolling 2d6...
Original roll: [2, 2]
After advantage: [2, 2]
Fighter 3 did 13 damage
Rolling 1d4...
Original roll: [2]
After advantage: [2]
After exploding: [2]
Fighter 3 rolls to attack (1d4+0, adv=0): [2]
Attacker hit = True
Fighter 3 did 2 damage
Rolling 1d8...
Original roll: [2, 3]
After advantage: [2]
After exploding: [2]
Fighter 3 rolls to attack (1d8+3, adv=-1): [2]
Attacker hit = True
Fighter 3 did 5 damage
Fighter 3 total damage this round: 20
20

WIZARD


In [62]:

print("\nGREAT_WEAPON")
fighter.attack_dice = "2d6"
print(get_damage_for_round(fighter, monster, NimbleGreatWeaponFighterAttack(advantage=0, gwm_advantage_threshold=0, use_total_damage=True), NimbleMonsterLightArmorDefense(), NimbleDiceRoller(), print_results=True))



GREAT_WEAPON
Rolling 2d6...
Original roll: [1, 6]
After advantage: [1, 6]
After exploding: [1, 6]
Fighter 3 rolls to attack (2d6+3, adv=0): [1, 6]
Great Weapons Fighter attack
Rolling 2d6...
Original roll: [1, 5]
After advantage: [1, 5]
After exploding: [1, 5]
Great Weapons Fighter hit
Fighter 3 did 16 damage
Rolling 2d6...
Original roll: [3, 4, 3]
After advantage: [3, 3]
After exploding: [3, 3]
Fighter 3 rolls to attack (2d6+3, adv=-1): [3, 3]
Normal attack
Attacker hit = True
Fighter 3 did 9 damage
Fighter 3 total damage this round: 25
25


In [63]:
class NimbleMonsterMediumArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls)

        if was_crit:
            damage += damage_modifier
        
        return damage

print(get_damage_for_round(fighter, monster, NimbleStandardAttack(advantage=0), NimbleMonsterMediumArmorDefense(), NimbleDiceRoller(), print_results=True))

Rolling 2d6...
Original roll: [1, 4]
After advantage: [1, 4]
After exploding: [1, 4]
Fighter 3 rolls to attack (2d6+3, adv=0): [1, 4]
Attacker hit = False
Fighter 3 did 0 damage
Rolling 2d6...
Original roll: [3, 3, 4]
After advantage: [3, 3]
After exploding: [3, 3]
Fighter 3 rolls to attack (2d6+3, adv=-1): [3, 3]
Attacker hit = True
Fighter 3 did 6 damage
Fighter 3 total damage this round: 6
6


In [64]:
class NimbleMonsterHeavyArmorDefense(DefenseProcessor):
    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls) + damage_modifier
        
        if not was_crit:
            damage = math.ceil(damage / 2)
        
        return damage
        
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterHeavyArmorDefense(), NimbleDiceRoller(), print_results=True))

Rolling 2d6...
Original roll: [2, 6]
After advantage: [2, 6]
After exploding: [2, 6]
Fighter 3 rolls to attack (2d6+3, adv=0): [2, 6]
Attacker hit = True
Fighter 3 did 6 damage
Rolling 2d6...
Original roll: [4, 6, 1]
After advantage: [4, 1]
After exploding: [4, 1]
Fighter 3 rolls to attack (2d6+3, adv=-1): [4, 1]
Attacker hit = True
Fighter 3 did 4 damage
Fighter 3 total damage this round: 10
10


In [65]:
class NimbleMonsterArmorDefense(DefenseProcessor):
    def __init__(self, monster):
        if monster.ac < 14:
            self.armor_processor = NimbleMonsterLightArmorDefense()
        elif monster.ac < 18:
            self.armor_processor = NimbleMonsterMediumArmorDefense()
        else:
            self.armor_processor = NimbleMonsterHeavyArmorDefense()

    def start_of_round(self):
        self.armor_processor.start_of_round()

    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        return self.armor_processor.did_attack_hit(attacker, defender, attack_dice, print_results)

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        return self.armor_processor.defend_against_attack(defender, attacker, die_rolls, damage_modifier, was_crit, print_results)

monster.ac = 12
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 15
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 19
print(get_damage_for_round(fighter, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))


monster.ac = 12
print(get_damage_for_round(fighter, monster, NimbleRogueAttack("1d4", "2d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 15
print(get_damage_for_round(fighter, monster, NimbleRogueAttack("1d4", "2d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))
print()

monster.ac = 19
print(get_damage_for_round(fighter, monster, NimbleRogueAttack("1d4", "2d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), print_results=True))

Rolling 2d6...
Original roll: [6, 5]
After advantage: [6, 5]
After exploding: [6, 5, 2]
Fighter 3 rolls to attack (2d6+3, adv=0): [6, 5, 2]
Attacker hit = True
Fighter 3 did 16 damage
Rolling 2d6...
Original roll: [4, 4, 5]
After advantage: [4, 4]
After exploding: [4, 4]
Fighter 3 rolls to attack (2d6+3, adv=-1): [4, 4]
Attacker hit = True
Fighter 3 did 11 damage
Fighter 3 total damage this round: 27
27

Rolling 2d6...
Original roll: [5, 1]
After advantage: [5, 1]
After exploding: [5, 1]
Fighter 3 rolls to attack (2d6+3, adv=0): [5, 1]
Attacker hit = True
Fighter 3 did 6 damage
Rolling 2d6...
Original roll: [4, 2, 6]
After advantage: [4, 2]
After exploding: [4, 2]
Fighter 3 rolls to attack (2d6+3, adv=-1): [4, 2]
Attacker hit = True
Fighter 3 did 6 damage
Fighter 3 total damage this round: 12
12

Rolling 2d6...
Original roll: [3, 4]
After advantage: [3, 4]
After exploding: [3, 4]
Fighter 3 rolls to attack (2d6+3, adv=0): [3, 4]
Attacker hit = True
Fighter 3 did 5 damage
Rolling 2d6...


In [66]:
class NimbleMonsterAttack(AttackProcessor):
    def perform_attacks(self, attacker, defender, defense_processor, dice_roller, print_results=False):
        debug_output = []
        
        total_damage = 0

        # Process normal attacks
        for _ in range(attacker.num_attacks):
            damage = self.perform_single_attack(attacker, defender, defense_processor, dice_roller, 0, print_results)
            total_damage += damage
            
        if print_results:
            print(f'{attacker.name} total damage this round: {total_damage}')

        return total_damage

    def perform_single_attack(self, attacker, defender, defense_processor, dice_roller, advantage, print_results=False):
        die_rolls = dice_roller.roll(attacker.attack_dice, advantage, print_results)

        if print_results:
            print(f'{attacker.name} rolls to attack ({attacker.attack_dice}+{attacker.damage_modifier}, adv={advantage}): {die_rolls}')

        hit = defense_processor.did_attack_hit(attacker, defender, die_rolls, print_results)

        was_crit = is_max_die_value(attacker.attack_dice, die_rolls[0])
        if hit:
            damage = defense_processor.defend_against_attack(defender, attacker, die_rolls, attacker.damage_modifier, was_crit, print_results)
        else:
            damage = 0

        if print_results:
            print(f'{attacker.name} did {damage} damage')

        return damage

In [67]:
class NimblePlayerDefense(DefenseProcessor):
    def __init__(self):
        self.has_blocked = False

    def start_of_round(self):
        self.has_blocked = False

    def did_attack_hit(self, attacker, defender, attack_dice, print_results = False) -> bool:
        did_hit = attack_dice[0] > 1

        if print_results:
            print(f'Attacker hit = {did_hit}')
        
        return did_hit

    def defend_against_attack(self, defender, attacker, die_rolls, damage_modifier, was_crit, print_results = False):
        damage = sum(die_rolls) + damage_modifier
        
        if not self.has_blocked:
            damage_reduction = defender.ac - 8

            if print_results:
                print(f'{defender.name} blocks {damage_reduction} damage')
            
            damage = max(0, damage - damage_reduction)
            self.has_blocked = True
        
        return damage

print(get_damage_for_round(monster, fighter, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller(), print_results=True))

Rolling 1d10...
Original roll: [4]
After advantage: [4]
After exploding: [4]
Light Armor Monster 3 rolls to attack (1d10+4, adv=0): [4]
Attacker hit = True
Fighter 3 blocks 10 damage
Light Armor Monster 3 did 0 damage
Rolling 1d10...
Original roll: [1]
After advantage: [1]
After exploding: [1]
Light Armor Monster 3 rolls to attack (1d10+4, adv=0): [1]
Attacker hit = False
Light Armor Monster 3 did 0 damage
Light Armor Monster 3 total damage this round: 0
0


In [68]:
def get_dpr(attacker, defender, attack_processor, defense_processor, dice_roller, num_simulations=100000, print_results=False):
    average = 0
    
    results = [get_damage_for_round(attacker, defender, attack_processor, defense_processor, dice_roller, print_results) for _ in range(num_simulations)]
    if print_results:
        print("\n")
    return mean(results), stdev(results)


In [69]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Fighter", hp=40, ac=12, num_attacks=2, attack_dice="1d6", damage_modifier=7)
monster = Character(name="Monster", hp=84, ac=20, num_attacks=2, attack_dice="3d8", damage_modifier=1)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr, player_dpr_stdev = get_dpr(player, monster, NimbleStandardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller())
player_turns = monster.hp / player_dpr
monster_dpr, monster_dpr_stdev = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller())
monster_turns = player.hp / monster_dpr

print("FIGHTER")
print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))


FIGHTER
Player DPR: 17.13867
Monster DPR: 23.61725
Player Turns to Win: 4.901197117395924
Monster Turns to Win: 1.6936772909631732


In [70]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Rogue 9", hp=46, ac=17, num_attacks=1, attack_dice="1d6", damage_modifier=6.5)
monster = Character(name="Monster", hp=84, ac=20, num_attacks=2, attack_dice="3d8", damage_modifier=1)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr, player_drp_stdev = get_dpr(player, monster, NimbleRogueAttack("1d6", "5d6"), NimbleMonsterArmorDefense(monster), NimbleDiceRoller())
player_turns = monster.hp / player_dpr
monster_dpr, monster_dpr_stdev = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller())
monster_turns = player.hp / monster_dpr

print("ROGUE")
print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))


ROGUE
Player DPR: 23.72837
Monster DPR: 18.74992
Player Turns to Win: 3.5400661739512658
Monster Turns to Win: 2.453343800933551


In [71]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Wizard 3", hp=40, ac=12, num_attacks=1, attack_dice="2d10", damage_modifier=0)
monster = Character(name="Monster", hp=84, ac=12, num_attacks=2, attack_dice="3d8", damage_modifier=1)
# END PLAYER AND MONSTER DEFINITIONS

player_dpr, _ = get_dpr(player, monster, NimbleWizardAttack(), NimbleMonsterArmorDefense(monster), NimbleDiceRoller())
player_turns = monster.hp / player_dpr
monster_dpr, _ = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller())
monster_turns = player.hp / monster_dpr

print("WIZARD")
print("Player DPR: " + str(player_dpr))
print("Monster DPR: " + str(monster_dpr))
print("Player Turns to Win: " + str(player_turns))
print("Monster Turns to Win: " + str(monster_turns))

WIZARD
Player DPR: 24.95506
Monster DPR: 23.57429
Player Turns to Win: 3.3660508129413436
Monster Turns to Win: 1.6967637201374888


In [72]:
# BEGIN PLAYER AND MONSTER DEFINITIONS
player = Character(name="Fighter", hp=40, ac=12, num_attacks=2, attack_dice="2d6", damage_modifier=7)
monster = Character(name="Monster", hp=84, ac=12, num_attacks=2, attack_dice="3d8", damage_modifier=1)
# END PLAYER AND MONSTER DEFINITIONS

def get_gwm_stats(player, monster):
    num_simulations=100000
    print_results=False
    gwm_advantage_threshold=0
    
    print(f"GREAT WEAPON MASTER ({player.num_attacks}, {player.attack_dice}+{player.damage_modifier}) vs. AC {monster.ac}")
    player_dpr_base, player_dpr_base_stdev = get_dpr(player, monster, NimbleStandardAttack(advantage=0), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), num_simulations, print_results)
    print("Player DPR Normal: " + str(player_dpr_base))# + "; stdev = " + str(player_dpr_base_stdev))
    player_dpr_gwm, player_dpr_gwm_stdev = get_dpr(player, monster, NimbleGreatWeaponFighterAttack(0, gwm_advantage_threshold), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), num_simulations, print_results)
    print("Player DPR GWM (fail on 4): " + str(player_dpr_gwm))# + "; stdev = " + str(player_dpr_gwm_stdev))
    player_dpr_gwm_easier, player_dpr_gwm_easier_stdev = get_dpr(player, monster, NimbleGreatWeaponFighterAttack(0, gwm_advantage_threshold, use_total_damage=False, use_easier=True), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), num_simulations, print_results)
    print("Player DPR GWM (easier 2d6): " + str(player_dpr_gwm_easier))# + "; stdev = " + str(player_dpr_gwm_easier))
    player_dpr_gwm_total, player_dpr_gwm_total_stdev = get_dpr(player, monster, NimbleGreatWeaponFighterAttack(0, gwm_advantage_threshold, use_total_damage=True), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), num_simulations, print_results)
    print("Player DPR GWM (all > 4): " + str(player_dpr_gwm_total))# + "; stdev = " + str(player_dpr_gwm_total))
#    player_dpr_base_adv, player_dpr_base_adv_stdev = get_dpr(player, monster, NimbleStandardAttack(advantage=1), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), num_simulations, print_results)
#    print("Player DPR Normal w/ advantage: " + str(player_dpr_base_adv))# + "; stdev = " + str(player_dpr_base_adv_stdev))
#    player_dpr_gwm_adv, player_dpr_gwm_adv_stdev = get_dpr(player, monster, NimbleGreatWeaponFighterAttack(1, gwm_advantage_threshold), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), num_simulations, print_results)
#    print("Player DPR GWM w/ advantage (old): " + str(player_dpr_gwm_adv))# + "; stdev = " + str(player_dpr_gwm_adv_stdev))
#    player_dpr_gwm_adv_total, player_dpr_gwm_adv_total_stdev = get_dpr(player, monster, NimbleGreatWeaponFighterAttack(1, gwm_advantage_threshold, use_total_damage=True), NimbleMonsterArmorDefense(monster), NimbleDiceRoller(), num_simulations, print_results)
#    print("Player DPR GWM w/ advantage (new): " + str(player_dpr_gwm_adv_total))# + "; stdev = " + str(player_dpr_gwm_adv_total_stdev))
    # monster_dpr = get_dpr(monster, player, NimbleMonsterAttack(), NimblePlayerDefense(), NimbleDiceRoller())
    # monster_turns = player.hp / monster_dpr
    

for monster_ac in [12,16,20]:
    monster.ac = monster_ac
    for attack_dice in ["2d6", "1d10", "1d12"]:
        player.attack_dice = attack_dice
        get_gwm_stats(player, monster)
        print()

GREAT WEAPON MASTER (2, 2d6+7) vs. AC 12
Player DPR Normal: 35.42098
Player DPR GWM (fail on 4): 27.13373
Player DPR GWM (easier 2d6): 34.43957
Player DPR GWM (all > 4): 37.27681

GREAT WEAPON MASTER (2, 1d10+7) vs. AC 12
Player DPR Normal: 34.01991
Player DPR GWM (fail on 4): 35.35364
Player DPR GWM (easier 2d6): 35.34999
Player DPR GWM (all > 4): 35.34149

GREAT WEAPON MASTER (2, 1d12+7) vs. AC 12
Player DPR Normal: 37.13955
Player DPR GWM (fail on 4): 41.547
Player DPR GWM (easier 2d6): 41.65976
Player DPR GWM (all > 4): 41.54708

GREAT WEAPON MASTER (2, 2d6+7) vs. AC 16
Player DPR Normal: 21.18977
Player DPR GWM (fail on 4): 19.78952
Player DPR GWM (easier 2d6): 24.86369
Player DPR GWM (all > 4): 26.8029

GREAT WEAPON MASTER (2, 1d10+7) vs. AC 16
Player DPR Normal: 17.20041
Player DPR GWM (fail on 4): 22.67962
Player DPR GWM (easier 2d6): 22.79216
Player DPR GWM (all > 4): 22.74323

GREAT WEAPON MASTER (2, 1d12+7) vs. AC 16
Player DPR Normal: 19.64277
Player DPR GWM (fail on 4): 27